In [ ]:
#刪除資料夾
rm -rf /content/2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import os

In [ ]:
# 定義卷積區塊 (ConvBlock)，用於 U-Net 編碼器和解碼器中
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        # 兩層卷積+批量正規化+ReLU激活
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1), # 第一個卷積層，保持輸出尺寸與輸入相同
            nn.BatchNorm2d(out_channels),# 批量正規化，用於穩定訓練過程
            nn.ReLU(inplace=True),# ReLU 激活函數，添加非線性特徵
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
         # 前向傳播過程：將輸入 x 通過定義的卷積區塊
        return self.conv(x)


In [ ]:
# 定義 U-Net 網絡結構
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super().__init__()

        # 編碼器部分 (Encoder)
        self.enc1 = ConvBlock(in_channels, 64)  # 第一層卷積區塊，輸入通道為圖像的通道數
        self.enc2 = ConvBlock(64, 128)      # 第二層卷積區塊，將輸出通道增加到128
        self.enc3 = ConvBlock(128, 256)     # 第三層卷積區塊
        self.enc4 = ConvBlock(256, 512)     # 第四層卷積區塊

        # Bottleneck (瓶頸部分)：對特徵圖進一步處理，提取最深層的特徵
        self.bottleneck = ConvBlock(512, 512)  # 修改這裡，從1024改為512

        # 解碼器部分 (Decoder)：通過反卷積和跳連接逐步恢復圖像尺寸
        self.upconv4 = nn.ConvTranspose2d(512, 512, kernel_size=2, stride=2) #上采樣層，將特徵圖尺寸放大一倍
        self.dec4 = ConvBlock(1024, 256)  # 512 + 512 = 1024 輸入通道

        self.upconv3 = nn.ConvTranspose2d(256, 256, kernel_size=2, stride=2)
        self.dec3 = ConvBlock(512, 128)   # 256 + 256 = 512 輸入通道

        self.upconv2 = nn.ConvTranspose2d(128, 128, kernel_size=2, stride=2)
        self.dec2 = ConvBlock(256, 64)    # 128 + 128 = 256 輸入通道

        self.upconv1 = nn.ConvTranspose2d(64, 64, kernel_size=2, stride=2)
        self.dec1 = ConvBlock(128, 64)    # 64 + 64 = 128 輸入通道

        # 最終輸出層：將解碼器的輸出轉換為最終的分割結果
        self.final_conv = nn.Conv2d(64, out_channels, 1)
        # 池化層：用於編碼器部分，縮小特徵圖的尺寸
        self.pool = nn.MaxPool2d(2)

    def forward(self, x):
        # 編碼過程
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))

        # Bottleneck：對編碼器最後一層的輸出進行進一步處理
        bottleneck = self.bottleneck(self.pool(enc4))

        # 解碼過程並添加跳連接
        dec4 = self.upconv4(bottleneck)#上采樣
        dec4 = torch.cat([dec4, enc4], dim=1)# 跳連接，將上采樣的輸出與編碼器對應層的輸出拼接
        dec4 = self.dec4(dec4) # 卷積區塊

        dec3 = self.upconv3(dec4)
        dec3 = torch.cat([dec3, enc3], dim=1)
        dec3 = self.dec3(dec3)

        dec2 = self.upconv2(dec3)
        dec2 = torch.cat([dec2, enc2], dim=1)
        dec2 = self.dec2(dec2)

        dec1 = self.upconv1(dec2)
        dec1 = torch.cat([dec1, enc1], dim=1)
        dec1 = self.dec1(dec1)

        return self.final_conv(dec1)


In [ ]:
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.utils import plot_model

# 安裝必要套件
import subprocess
import sys

def install_requirements():
    packages = ['pydot', 'graphviz']
    for package in packages:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

def build_unet(input_shape=(256, 256, 3), out_channels=1):
    inputs = Input(shape=input_shape)

    # 編碼器部分 (Encoder)
    enc1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(enc1)

    enc2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(enc2)

    enc3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(enc3)

    enc4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    pool4 = MaxPooling2D(pool_size=(2, 2))(enc4)

    # Bottleneck（瓶頸層）
    bottleneck = Conv2D(512, 3, activation='relu', padding='same')(pool4)

    # 解碼器部分 (Decoder)
    up4 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(bottleneck)
    concat4 = concatenate([up4, enc4], axis=3)
    dec4 = Conv2D(256, 3, activation='relu', padding='same')(concat4)

    up3 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(dec4)
    concat3 = concatenate([up3, enc3], axis=3)
    dec3 = Conv2D(128, 3, activation='relu', padding='same')(concat3)

    up2 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(dec3)
    concat2 = concatenate([up2, enc2], axis=3)
    dec2 = Conv2D(64, 3, activation='relu', padding='same')(concat2)

    up1 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(dec2)
    concat1 = concatenate([up1, enc1], axis=3)
    dec1 = Conv2D(64, 3, activation='relu', padding='same')(concat1)

    # 最終輸出層
    outputs = Conv2D(out_channels, 1, activation='sigmoid')(dec1)

    model = Model(inputs=inputs, outputs=outputs)
    return model

def plot_unet_structure():
    try:
        # 安裝必要的套件
        install_requirements()

        # 創建模型
        model = build_unet()

        # 生成模型圖
        plot_model(
            model,
            to_file='unet_structure.png',
            show_shapes=True,
            show_layer_names=True,
            rankdir='TB',
            dpi=96,
            expand_nested=True
        )
        print("模型結構圖已保存為 'unet_structure.png'")

    except Exception as e:
        print(f"生成圖形時出錯: {str(e)}")
        print("請確保已安裝 graphviz:")
        print("Ubuntu/Debian: sudo apt-get install graphviz")
        print("Mac: brew install graphviz")
        print("Windows: 從 graphviz.org 下載安裝包")

if __name__ == '__main__':
    plot_unet_structure()


模型結構圖已保存為 'unet_structure.png'


In [ ]:
# 自定義資料集，用於加載圖像和遮罩
class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform

        # 檢查圖像和遮罩資料夾是否存在
        if not os.path.exists(image_dir):
            raise ValueError(f"圖像目錄不存在: {image_dir}")
        if not os.path.exists(mask_dir):
            raise ValueError(f"遮罩目錄不存在: {mask_dir}")

        self.images = [f for f in os.listdir(image_dir) if f.endswith('.png')]
        self.valid_pairs = []

        # 檢查每個圖像對應的遮罩是否存在
        for img_name in self.images:
            base_name = img_name.rsplit('.', 1)[0]
            mask_name = f"{base_name}_labeled.png"#標籤檔名
            mask_path = os.path.join(mask_dir, mask_name)

            if os.path.exists(mask_path):
                self.valid_pairs.append((img_name, mask_name))
                print(f"找到有效配對 - 圖像: {img_name}, 遮罩: {mask_name}")

        if not self.valid_pairs:
            raise ValueError("找不到有效的圖像-遮罩配對")

        print(f"找到 {len(self.valid_pairs)} 個有效的圖像-遮罩配對")

    def __len__(self):
        return len(self.valid_pairs)

    def __getitem__(self, idx):
        try:
            img_name, mask_name = self.valid_pairs[idx]
            image_path = os.path.join(self.image_dir, img_name)
            mask_path = os.path.join(self.mask_dir, mask_name)

            # 打開圖像和遮罩
            image = Image.open(image_path).convert('RGB')
            mask = Image.open(mask_path).convert('L')

            # 應用變換
            if self.transform:
                image = self.transform['image'](image)
                mask = self.transform['mask'](mask)

            return image, mask

        except Exception as e:
            print(f"加載圖像對 {img_name} 時出錯: {str(e)}")
            raise

In [ ]:
def calculate_accuracy(outputs, masks, threshold=0.5):
    """計算分割準確度"""
    with torch.no_grad():
        # 將輸出轉換為二值預測
        predicted = torch.sigmoid(outputs) > threshold
        masks = masks > threshold

        # 計算IoU (Intersection over Union)
        intersection = (predicted & masks).float().sum((1, 2, 3))#  Intersection (交集)：預測分割區域與真實分割區域重疊的部分。
        union = (predicted | masks).float().sum((1, 2, 3))#Union (聯集)：預測分割區域和真實分割區域的總和，不重複計算重疊的部分。
        iou = (intersection + 1e-6) / (union + 1e-6)

        # 計算像素準確度:比較預測結果與真實標籤，計算每個像素的準確性，然後取平均
        pixel_accuracy = (predicted == masks).float().mean((1, 2, 3))

        return iou.mean().item(), pixel_accuracy.mean().item()


In [ ]:
def train_model(model, train_loader, criterion, optimizer, device, num_epochs=100):
    model.train()# 設置模型為訓練模式

    for epoch in range(num_epochs):
        running_loss = 0.0 # 初始化每個 epoch 的累積損失

        for batch_idx, (images, masks) in enumerate(train_loader):
            images = images.to(device)
            masks = masks.to(device)

            optimizer.zero_grad()# 清除上一次梯度
            outputs = model(images)  # 將圖像輸入模型獲取輸出
            loss = criterion(outputs, masks) # 計算損失
            loss.backward()# 反向傳播計算梯度
            optimizer.step()# 更新模型參數

            running_loss += loss.item()# 累加損失

            if batch_idx % 3 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')
        # 計算並打印該 epoch 的平均損失
        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {epoch_loss:.4f}')

        # 每2個epoch計算並輸出準確度
        if (epoch + 1) % 2 == 0:
            model.eval()# 設置模型為評估模式
            total_iou = 0.0
            total_pixel_acc = 0.0
            num_batches = 0

            with torch.no_grad():# 關閉梯度計算，節省記憶體(在評估階段不需要計算梯度)
                for images, masks in train_loader:
                    images = images.to(device)
                    masks = masks.to(device)
                    outputs = model(images)

                    iou, pixel_acc = calculate_accuracy(outputs, masks)
                    total_iou += iou
                    total_pixel_acc += pixel_acc
                    num_batches += 1

            # 計算平均 IoU 和像素準確度
            avg_iou = total_iou / num_batches
            avg_pixel_acc = total_pixel_acc / num_batches
            print(f'Epoch [{epoch+1}/{num_epochs}] Metrics:')
            print(f'Average IoU: {avg_iou:.4f}')
            print(f'Average Pixel Accuracy: {avg_pixel_acc:.4f}')

            model.train()# 恢復模型到訓練模式

In [ ]:
def main():
    # 設置設備為 GPU (cuda) 如果可用，否則使用 CPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"使用設備: {device}")

    image_dir = "/content/1"
    mask_dir = "/content/2"

    model = UNet(in_channels=3, out_channels=1).to(device)
    # 設置損失函數為二元交叉熵損失，適用於二分類問題
    criterion = nn.BCEWithLogitsLoss()
    # 使用 Adam 優化器，學習率為 1e-4
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    transforms_dict = {
        'image': transforms.Compose([
            transforms.Resize((256, 256)), # 將圖像調整到 256x256
            transforms.ToTensor(),# 將圖像轉換為張量
            transforms.Normalize(mean=[0.485, 0.456, 0.406], # 正規化圖像
                               std=[0.229, 0.224, 0.225])
        ]),
        'mask': transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor()
        ])
    }

    try:
        dataset = CustomDataset(image_dir, mask_dir, transform=transforms_dict)
        train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

        print("開始訓練...")
        train_model(model, train_loader, criterion, optimizer, device, num_epochs=20)

        torch.save(model.state_dict(), 'unet_model.pth')
        print("模型已保存為 'unet_model.pth'")

    except Exception as e:
        print(f"程序執行出錯: {str(e)}")
        raise

if __name__ == '__main__':
    main()

使用設備: cuda
找到有效配對 - 圖像: 0006R0_f02130.png, 遮罩: 0006R0_f02130_labeled.png
找到有效配對 - 圖像: 0001TP_010350.png, 遮罩: 0001TP_010350_labeled.png
找到有效配對 - 圖像: 0006R0_f01530.png, 遮罩: 0006R0_f01530_labeled.png
找到有效配對 - 圖像: 0006R0_f02460.png, 遮罩: 0006R0_f02460_labeled.png
找到有效配對 - 圖像: 0006R0_f00930.png, 遮罩: 0006R0_f00930_labeled.png
找到有效配對 - 圖像: 0006R0_f01800.png, 遮罩: 0006R0_f01800_labeled.png
找到有效配對 - 圖像: 0001TP_010320.png, 遮罩: 0001TP_010320_labeled.png
找到有效配對 - 圖像: 0006R0_f02730.png, 遮罩: 0006R0_f02730_labeled.png
找到有效配對 - 圖像: 0006R0_f02010.png, 遮罩: 0006R0_f02010_labeled.png
找到有效配對 - 圖像: 0006R0_f01830.png, 遮罩: 0006R0_f01830_labeled.png
找到有效配對 - 圖像: 0001TP_010020.png, 遮罩: 0001TP_010020_labeled.png
找到有效配對 - 圖像: 0001TP_009450.png, 遮罩: 0001TP_009450_labeled.png
找到有效配對 - 圖像: 0006R0_f01470.png, 遮罩: 0006R0_f01470_labeled.png
找到有效配對 - 圖像: 0006R0_f01860.png, 遮罩: 0006R0_f01860_labeled.png
找到有效配對 - 圖像: 0006R0_f02640.png, 遮罩: 0006R0_f02640_labeled.png
找到有效配對 - 圖像: 0006R0_f01230.png, 遮罩: 0006R0_f01230_labeled.p

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from torchvision.utils import save_image
import os

class UNetPredictor:
    def __init__(self, model_path, device=None):
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # 載入模型架構
        self.model = UNet(in_channels=3, out_channels=1)

        # 載入訓練好的權重
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.to(self.device)
        self.model.eval()

        # 設定圖片預處理
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

    def predict(self, image_path, output_path):
        """
        預測單張圖片的遮罩

        Args:
            image_path (str): 輸入圖片的路徑
            output_path (str): 輸出遮罩的保存路徑
        """
        try:
            # 載入並預處理圖片
            image = Image.open(image_path).convert('RGB')
            original_size = image.size

            # 轉換圖片格式
            image_tensor = self.transform(image).unsqueeze(0)
            image_tensor = image_tensor.to(self.device)

            # 進行預測
            with torch.no_grad():
                output = self.model(image_tensor)
                prediction = torch.sigmoid(output)

            # 將預測結果轉換為二值遮罩 (threshold = 0.5)
            binary_mask = (prediction > 0.5).float()

            # 保存預測結果
            save_image(binary_mask, output_path)

            print(f"預測完成，結果已保存至: {output_path}")

        except Exception as e:
            print(f"預測過程發生錯誤: {str(e)}")
            raise

def predict_batch(model_path, input_dir, output_dir):
    """
    批次預測多張圖片

    Args:
        model_path (str): 模型檔案路徑
        input_dir (str): 輸入圖片目錄
        output_dir (str): 輸出遮罩目錄
    """
    # 創建輸出目錄
    os.makedirs(output_dir, exist_ok=True)

    # 初始化預測器
    predictor = UNetPredictor(model_path)

    # 獲取所有圖片檔案
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    print(f"開始處理 {len(image_files)} 張圖片...")

    # 對每張圖片進行預測
    for image_file in image_files:
        input_path = os.path.join(input_dir, image_file)
        output_path = os.path.join(output_dir, f"{os.path.splitext(image_file)[0]}_mask.png")

        print(f"處理圖片: {image_file}")
        predictor.predict(input_path, output_path)

if __name__ == '__main__':
    # 使用範例
    model_path = '/content/unet_model.pth'  # 訓練好的模型路徑
    input_dir = '/content/1'      # 測試圖片目錄
    output_dir = '/content/3'      # 預測結果輸出目錄

    predict_batch(model_path, input_dir, output_dir)

<ipython-input-10-df6264d38083>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path, map_location=self.device))


開始處理 64 張圖片...
處理圖片: 0006R0_f02130.png
預測完成，結果已保存至: /content/3/0006R0_f02130_mask.png
處理圖片: 0001TP_010350.png
預測完成，結果已保存至: /content/3/0001TP_010350_mask.png
處理圖片: 0006R0_f01530.png
預測完成，結果已保存至: /content/3/0006R0_f01530_mask.png
處理圖片: 0006R0_f02460.png
預測完成，結果已保存至: /content/3/0006R0_f02460_mask.png
處理圖片: 0006R0_f00930.png
預測完成，結果已保存至: /content/3/0006R0_f00930_mask.png
處理圖片: 0006R0_f01800.png
預測完成，結果已保存至: /content/3/0006R0_f01800_mask.png
處理圖片: 0001TP_010320.png
預測完成，結果已保存至: /content/3/0001TP_010320_mask.png
處理圖片: 0006R0_f02730.png
預測完成，結果已保存至: /content/3/0006R0_f02730_mask.png
處理圖片: 0006R0_f02010.png
預測完成，結果已保存至: /content/3/0006R0_f02010_mask.png
處理圖片: 0006R0_f01830.png
預測完成，結果已保存至: /content/3/0006R0_f01830_mask.png
處理圖片: 0001TP_010020.png
預測完成，結果已保存至: /content/3/0001TP_010020_mask.png
處理圖片: 0001TP_009450.png
預測完成，結果已保存至: /content/3/0001TP_009450_mask.png
處理圖片: 0006R0_f01470.png
預測完成，結果已保存至: /content/3/0006R0_f01470_mask.png
處理圖片: 0006R0_f01860.png
預測完成，結果已保存至: /content/3/0006R0_f01860_

In [ ]:
import torch
import torchvision.transforms as transforms
import cv2
import numpy as np
import os
from PIL import Image

class UNetPredictor:
    def __init__(self, model_path, device=None):
        """
        初始化 UNet 預測器，載入模型權重和設置設備
        """
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # 載入 U-Net 模型架構
        self.model = UNet(in_channels=3, out_channels=1)

        # 載入訓練好的模型權重
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.to(self.device)
        self.model.eval()

        # 設定圖片預處理方式
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),  # 調整圖片大小
            transforms.ToTensor(),  # 轉換為 Tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406],  # 圖片標準化
                                 std=[0.229, 0.224, 0.225])
        ])

    def predict_frame(self, frame):
        """
        預測單一幀的遮罩
        frame: 單幀影像
        return: 預測的二值遮罩 (0 或 1)
        """
        # 將 BGR 圖片轉換為 RGB 格式
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # 預處理並轉換為 tensor
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)

        # 不進行梯度計算，預測結果
        with torch.no_grad():
            output = self.model(image_tensor)
            prediction = torch.sigmoid(output)  # 使用 sigmoid 來獲得概率值
            binary_mask = (prediction > 0.5).float().cpu().numpy().squeeze()  # 轉換為二值遮罩 (0 或 1)

        return binary_mask  # 這裡是 0 (黑色) 或 1 (白色) 的二值遮罩

def process_video(model_path, video_path, final_mask_path, frame_step=15, black_threshold=3):
    """
    讀取影片，每 `frame_step` 幀預測一次，並根據每個像素的黑色次數決定最終遮罩
    model_path: 訓練好的模型路徑
    video_path: 影片路徑
    final_mask_path: 最終遮罩輸出路徑
    frame_step: 每幾幀處理一次
    black_threshold: 像素點必須在多少幀中為黑色，最終遮罩才變黑
    """
    predictor = UNetPredictor(model_path)  # 初始化預測器
    cap = cv2.VideoCapture(video_path)  # 讀取影片

    if not cap.isOpened():
        print("無法開啟影片")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))  # 取得影片 FPS
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 總幀數
    height, width = None, None
    frame_idx = 0

    black_pixel_count = None  # 記錄每個像素變黑的次數
    frame_count = 0  # 計算總共處理的幀數

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # 影片播放結束

        if height is None or width is None:
            height, width, _ = frame.shape
            black_pixel_count = np.zeros((height, width), dtype=np.uint16)  # 初始化黑色像素計數

        # **每 `frame_step` 幀處理一次**
        if frame_idx % frame_step == 0:
            mask = predictor.predict_frame(frame)  # 取得預測遮罩 (0 或 1)
            mask = cv2.resize(mask, (width, height))  # 調整遮罩大小回原解析度
            mask = (mask * 255).astype(np.uint8)  # 轉換為 0 (黑) 或 255 (白)

            # 更新黑色像素出現次數
            black_pixel_count[mask == 0] += 1

            frame_count += 1  # 記錄處理的幀數

        frame_idx += 1

    cap.release()  # 釋放影片資源

    # **根據黑色像素的出現次數決定最終遮罩**
    final_mask = np.ones((height, width), dtype=np.uint8) * 255  # 預設為全白
    final_mask[black_pixel_count >= black_threshold] = 0  # 若某像素在多於 `black_threshold` 幀中為黑色，最終該像素為黑色

    # 儲存最終遮罩
    cv2.imwrite(final_mask_path, final_mask)
    print(f"最終遮罩已儲存: {final_mask_path}, 影片共使用 {frame_count} 張影格進行處理")

if __name__ == '__main__':
    # 設定模型路徑、影片路徑和最終遮罩的輸出路徑
    model_path = '/content/unet_model.pth'
    video_path = '/content/下載.mp4'
    final_mask_path = '/content/final_mask.png'  # 儲存最終的遮罩圖片
    frame_step = 5  # 每 5 幀預測一次
    black_threshold = 4  # 若某像素在 4 幀中變黑，最終結果才變黑

    process_video(model_path, video_path, final_mask_path, frame_step, black_threshold)
